In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
def Unfreeze_Layers(pretrain, layer_list):
    pretrain.trainable = True
    for layer in  pretrain.layers:
        if layer.name in layer_list:
            layer.trainable = True
        else:
            layer.trainable = False
        
    for layer in pretrain.layers:
        print(layer.name, layer.trainable)
    print(len(pretrain.trainable_weights))

In [4]:
def Freeze_Pretrained_Base(pretrain, network):
    pretrain.trainable = False
    for layer in network.layers:
        print(layer.name, layer.trainable)
    print(len(network.trainable_weights))

In [5]:
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [6]:
def print_metrics(model_history):

    metrics = ['val_loss', 'val_acc', 'val_recall', 'val_precision', 'val_true_positives', 'val_true_negatives', 'val_false_positives', 'val_false_negatives']
    tp = model_history['val_true_positives'][-1]
    fp = model_history['val_false_positives'][-1]
    fn = model_history['val_false_negatives'][-1]
    f1 = tp/(tp+(0.5*(fp+fn)))
    for x in metrics:
        print(x+':',model_history[x][-1])
    print('F1 score:',f1)

In [48]:
# Set random state for numpy operations
from numpy.random import seed
seed(2)
# Set random state for tensorflow operations
from tensorflow.random import set_seed
set_seed(3)
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import cv2
import PIL

In [8]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.2]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.2]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    classes={'no_tumor': 0,
                                            'glioma_tumor':1,
                                            'meningioma_tumor':2,
                                            'pituitary_tumor':3},
                                    target_size=(150,150),
                                    batch_size=2700,# number of training images
                                    seed=42,
                                    class_mode='categorical',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Testing',
                                    classes={'no_tumor': 0,
                                            'glioma_tumor':1,
                                            'meningioma_tumor':2,
                                            'pituitary_tumor':3},
                                    target_size=(150,150),
                                    batch_size=394,# number of images
                                    seed=42,
                                    class_mode='categorical')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    classes={'no_tumor': 0,
                                            'glioma_tumor':1,
                                            'meningioma_tumor':2,
                                            'pituitary_tumor':3},
                                    target_size=(150,150),
                                    batch_size=170,# number of images
                                    seed=42,
                                    class_mode='categorical',
                                    subset='validation')
# First run-throughs were not done with a random seed, so model analysis may be slightly different from what will be the 
# actual numbers after running models with the random seed.

In [9]:
# # Creating variables to contain image vectors and labels for the different training sets
train_img, train_lab = next(train_generator)
test_img, test_lab = next(test_generator)
val_img, val_lab = next(val_generator)

## **Using Pre-Trained VGG-19 Weights**

In [10]:
from keras.applications.vgg19 import VGG19
cnn_vgg = VGG19(weights='imagenet',
               include_top=False,
               input_shape=(150,150,3))

In [12]:
# Making early stop for model
pre_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='pretrained_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [13]:
cnn_vgg.summary()

In [14]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
pretrained = keras.Sequential()
pretrained.add(cnn_vgg)
pretrained.add(layers.Flatten())
pretrained.add(layers.Dense(128, activation='relu'))
pretrained.add(layers.Dense(4, activation='softmax'))

In [15]:
Freeze_Pretrained_Base(cnn_vgg, pretrained)

In [16]:

pretrained.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

pretrained_results = pretrained.fit(x=train_img, y=train_lab,
                                              batch_size = 32,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= pre_early,
                                         validation_data=(test_img, test_lab),
                                        validation_steps = 394//32+1)

In [17]:
visualize_training_results(pretrained_results)

In [18]:
pretrained.evaluate(test_img, test_lab)

 loss: 1.266525387763977,\
 acc: 0.5634517669677734,\
 recall: 0.5025380849838257,\
 precision: 0.5892857313156128,\
 tp: 198.0,\
 tn: 1044.0,\
fp: 138.0,\
fn: 196.0

## **Unfreezing an outer Layer of the Pretrained Network**

In [19]:
# Making early stop for model
b5c1_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='b5_c1_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [20]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5_c1 = keras.Sequential()
b5_c1.add(cnn_vgg)
b5_c1.add(layers.Flatten())
b5_c1.add(layers.Dense(128, activation='relu'))
b5_c1.add(layers.Dense(4, activation='softmax'))

In [21]:
# freezing everything 
Freeze_Pretrained_Base(cnn_vgg, b5_c1)

In [22]:
# Unfreezing the last layer of the pretrained CNN
un_b5c1 = ['block5_conv1']
Unfreeze_Layers(cnn_vgg, un_b5c1)

In [23]:
b5_c1.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5_c1_results = b5_c1.fit(x=train_img, y=train_lab,
                                         batch_size=32,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= b5c1_early,
                                         validation_data=(test_img, test_lab),
                                        validation_steps=394//32+1)

In [24]:
visualize_training_results(b5_c1_results)

In [25]:
b5_c1.evaluate(test_img, test_lab)

## **Adding Dropout layers to VGG-19 pretrained network (one layer unfrozen)**

In [26]:
# Making early stop for model
vgg_drop_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='vgg_drop_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [27]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
vgg_drop = keras.Sequential()
vgg_drop.add(cnn_vgg)
vgg_drop.add(layers.Flatten())
vgg_drop.add(layers.Dropout(0.4))
vgg_drop.add(layers.Dense(128, activation='relu'))
vgg_drop.add(layers.Dropout(0.2))
vgg_drop.add(layers.Dense(4, activation='softmax'))

In [28]:
# Freeze all layers
Freeze_Pretrained_Base(cnn_vgg, vgg_drop)

In [29]:
# Unfreezing the last layer of the pretrained CNN
un_b5c1 = ['block5_conv1']
Unfreeze_Layers(cnn_vgg, un_b5c1)

In [30]:
vgg_drop.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

vgg_drop_results = vgg_drop.fit(x=train_img, y=train_lab,
                                            batch_size=32,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= vgg_drop_early,
                                         validation_data=(test_img, test_lab),
                                           validation_steps=394//32+1)

In [31]:
visualize_training_results(vgg_drop_results)

In [32]:
vgg_drop.evaluate(test_img, test_lab)

## **Unfreezing another layer of VGG Pretrained Network**

In [40]:
# Making early stop for model
b5_c1c2_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='b5c1c2_model.h5', monitor='val_acc',
                           save_best_only=True)]

In [34]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5_c1c2 = keras.Sequential()
b5_c1c2.add(cnn_vgg)
b5_c1c2.add(layers.Flatten())
b5_c1c2.add(layers.Dropout(0.4))
b5_c1c2.add(layers.Dense(128, activation='relu'))
b5_c1c2.add(layers.Dropout(0.2))
b5_c1c2.add(layers.Dense(4, activation='softmax'))

In [35]:
# Freeze all layers
Freeze_Pretrained_Base(cnn_vgg, b5_c1c2)

In [36]:
# Unfreezing the last layer of the pretrained CNN
un_b5c1c2 = ['block5_conv1', 'block5_conv2']
Unfreeze_Layers(cnn_vgg, un_b5c1c2)

In [44]:
b5_c1c2.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5_c1c2_results = b5_c1c2.fit(x=train_img, y=train_lab,
                                            batch_size=32,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= vgg_drop_early,
                                         validation_data=(test_img, test_lab))
                                           #validation_steps=394//32+1)

In [45]:
visualize_training_results(b5_c1c2_results)

In [46]:
b5_c1c2.evaluate(test_img, test_lab)

## **Implementing Learning Rate Reduction**

In [64]:
# Making early stop for model
red_c1c2_early = [EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            ModelCheckpoint(filepath='b5c1c2_model.h5', monitor='val_loss', save_best_only=True),
            ReduceLROnPlateau(patience=12, verbose=1)]

In [65]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
red_c1c2 = keras.Sequential()
red_c1c2.add(cnn_vgg)
red_c1c2.add(layers.Flatten())
red_c1c2.add(layers.Dropout(0.4))
red_c1c2.add(layers.Dense(128, activation='relu'))
red_c1c2.add(layers.Dropout(0.2))
red_c1c2.add(layers.Dense(4, activation='softmax'))

In [66]:
# Freeze all layers
Freeze_Pretrained_Base(cnn_vgg, red_c1c2)

In [67]:
# Unfreezing the last layer of the pretrained CNN
un_b5c1c2 = ['block5_conv1', 'block5_conv2']
Unfreeze_Layers(cnn_vgg, un_b5c1c2)

In [68]:
red_c1c2.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

red_c1c2_results = red_c1c2.fit(x=train_img, y=train_lab,
                                            batch_size=32,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=100,
                                        callbacks= red_c1c2_early,
                                         validation_data=(test_img, test_lab),
                                           validation_steps=394//32+1)

In [69]:
visualize_training_results(red_c1c2_results)

In [70]:
red_c1c2.evaluate(test_img, test_lab)

**Best Iteration!**\
loss: 1.5687031745910645,\
acc: 0.7461928725242615,\
recall: 0.7461928725242615,\
precision: 0.7577319741249084,\
tp: 294.0,\
tn: 1088.0,\
fp: 94.0,\
fn: 100.0

## **Adding Batch Normalization to VGG19 Pretrained Neural Network**

In [ ]:
# Making early stop for model
vgg_batch_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='vgg_batch_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
vgg_batch = keras.Sequential()
vgg_batch.add(cnn_vgg)
vgg_batch.add(layers.Flatten())
vgg_batch.add(layers.BatchNormalization())
vgg_batch.add(layers.Dropout(0.4))
vgg_batch.add(layers.Dense(128, activation='relu'))
vgg_batch.add(layers.BatchNormalization())
vgg_batch.add(layers.Dropout(0.2))
vgg_batch.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
Freeze_Pretrained_Base(cnn_vgg, vgg_batch)

In [ ]:
vgg_batch.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

vgg_batch_results = vgg_batch.fit(x=train_img, y=train_lab,
                                         steps_per_epoch=2700//32+1,
                                          batch_size=32,
                                         epochs=25,
                                        callbacks= vgg_batch_early,
                                         validation_data=(test_img, test_lab),
                                         validation_steps=394//32+1)

In [ ]:
visualize_training_results(vgg_batch_results)

In [ ]:
vgg_batch.evaluate(test_img, test_lab)

## **Adding Regularization VGG19 Pretrained Network**

In [ ]:
# Making early stop for model
reg_vgg_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='vgg_batch_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
reg_vgg = keras.Sequential()
reg_vgg.add(cnn_vgg)
reg_vgg.add(layers.Dropout(0.4))
reg_vgg.add(layers.Flatten())
reg_vgg.add(layers.Dense(128, activation='relu', kernel_regularizer=l2(l2=0.001)))
reg_vgg.add(layers.Dropout(0.2))
reg_vgg.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
Freeze_Pretrained_Base(cnn_vgg, reg_vgg)

In [ ]:
reg_vgg.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

reg_vgg_results = reg_vgg.fit(x=train_img, y=train_lab,
                              batch_size=32,
                              steps_per_epoch=2700//32+1,# number of samples / batch size
                              epochs=25,
                             callbacks= reg_vgg_early,
                            validation_data= (test_img, test_lab),
                            validation_steps = 394//32+1)

In [ ]:
visualize_training_results(reg_vgg_results)

In [ ]:
reg_vgg.evaluate(test_img, test_lab)

## **Changing Optimizer to SGD**

In [ ]:
sgd_mom = SGD(learning_rate = 0.001, momentum=0.9, nesterov=True)

In [ ]:
# Making early stop for model
sgd_vgg_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='vgg_sgd_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
sgd_vgg = keras.Sequential()
sgd_vgg.add(cnn_vgg)
sgd_vgg.add(layers.Dropout(0.4))
sgd_vgg.add(layers.Flatten())
sgd_vgg.add(layers.Dense(128, activation='relu'))
sgd_vgg.add(layers.Dropout(0.2))
sgd_vgg.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
Freeze_Pretrained_Base(cnn_vgg, sgd_vgg)

In [ ]:
sgd_vgg.compile(loss='categorical_crossentropy',
                optimizer= sgd_mom,
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

sgd_vgg_results = sgd_vgg.fit(x=train_img, y=train_lab,
                              batch_size=32,
                              steps_per_epoch=2700//32+1,# number of samples / batch size
                              epochs=25,
                             callbacks= sgd_vgg_early,
                            validation_data= (test_img, test_lab),
                            validation_steps = 394//32+1)

In [ ]:
visualize_training_results(sgd_vgg_results)

In [ ]:
sgd_vgg.evaluate(test_img, test_lab)

loss: 1.1398 - acc: 0.4833 - recall: 0.2544 - precision: 0.5687 - true_positives: 687.0000 - true_negatives: 7579.0000 - false_positives: 521.0000 - false_negatives: 2013.0000 - val_loss: 1.4112 - val_acc: 0.3909 - val_recall: 0.1218 - val_precision: 0.5106 - val_true_positives: 48.0000 - val_true_negatives: 1136.0000 - val_false_positives: 46.0000 - val_false_negatives: 346.0000

## **Using Pretrained Weights from VGG-16**

In [ ]:
from keras.applications.vgg16 import VGG16
vgg16 = VGG16(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
vgg16.summary()

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
vgg16_model = keras.Sequential()
vgg16_model.add(vgg16)
vgg16_model.add(layers.Flatten())
vgg16_model.add(layers.Dense(128, activation='relu'))
vgg16_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
vgg16.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in vgg16_model.layers:
    print(layer.name, layer.trainable)
    
print(len(vgg16_model.trainable_weights))

In [ ]:
vgg16_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])



In [ ]:
vgg16_model_results = vgg16_model.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 86% and a testing accuracy of 68%, with a training loss of 37% and a testing loss of 110%. Testing recall is 65%. Testing loss is slightly lower than the previous model using VGG19, but accuracy and recall is significantly lower, so it looks like using the VGG network with more layers is better. However, it is still a good idea to continue to try different pretrained networks.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
incep = InceptionV3(weights='imagenet',
               include_top=False,
               input_shape=(100,100,3))


In [ ]:
incep.summary()

In [ ]:
# Making early stop for model
incep_early = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='incep_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep_model = keras.Sequential()
incep_model.add(incep)
incep_model.add(layers.Flatten())
incep_model.add(layers.Dense(128, activation='relu'))
incep_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
Freeze_Pretrained_Base(incep, incep_model)

In [ ]:
incep_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep_model_results = incep_model.fit(x=train_img, y=train_lab,
                                      batch_size=32,
                                        steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= incep_early,
                                         validation_data=(test_img, test_lab),
                                     validation_steps=394//32+1)

In [ ]:
visualize_training_results(incep_model_results)

**Analysis of Model**

The epoch with the lowest testing loss has a training accuracy of 91% and a testing accuracy of 75%, with a training loss of 25% and a testing loss of 90%. Testing recall is around 73% This pretrained network resulted in the lowest testing loss yet, which is very promising; in the next few iterations I will try fine tuning this inception pretrained network.

## **Fine Tuning of Inception Network**
Adding class weights to account for class imbalance, adding batch normalization and dropout layer, unfreezing outer most convolution layer of Inception Network

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}
# Add class weights
# unfreeze outer layer
# decrease learning rate

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep2_model = keras.Sequential()
incep2_model.add(incep)
incep2_model.add(layers.Flatten())
incep2_model.add(layers.Dense(512, activation='relu'))
incep2_model.add(layers.BatchNormalization())
incep2_model.add(layers.Dropout(0.3))
incep2_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep2_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep2_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

incep2_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep2_model_results = incep2_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep2_model_results)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 94% and a testing accuracy of 77%, with a training loss of 16% and a testing loss of 105%. Testing recall is 69%. Accuracy between this iteration and the last is similar, but testing loss is about ten percentage points higher, so the model still requires some tuning. 

## **Adjusting Previus Model by Removing Class Weights, Removing Batch Normalization, moving the Dropout Layer, and Unfreezing more Layers**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep3_model = keras.Sequential()
incep3_model.add(incep)
incep3_model.add(layers.Dropout(0.5))
incep3_model.add(layers.Flatten())
incep3_model.add(layers.Dense(512, activation='relu'))
incep3_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.summary()

In [ ]:
unfreeze = ['conv2d_93', 'conv2d_85', 'conv2d_92', 'conv2d_91', 'conv2d_88', 'conv2d_87', 'conv2d_90', 'conv2d_86']

In [ ]:
Unfreeze_Layers(incep, unfreeze)

In [ ]:
incep3_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep3_model_results = incep3_model.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss has a training accuracy of 83% and a testing accuracy of 66%, with a training loss of 45% and a testing loss of 103%. Testing recall is 62%. These numbers are definitely worse than other iterations, so it looks removing the class weights and batch normalization wasn't the best idea. Additionally, maybe too many layers have been unfrozen. Overall, it seems that too many things were changed at once.

## **Putting back Batch Normalization, Adding another Dropout Layer**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep4_model = keras.Sequential()
incep4_model.add(incep)
incep4_model.add(layers.Dropout(0.4))
incep4_model.add(layers.BatchNormalization())
incep4_model.add(layers.Flatten())
incep4_model.add(layers.Dense(512, activation='relu'))
incep4_model.add(layers.Dropout(0.3))
incep4_model.add(layers.BatchNormalization())
incep4_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
Unfreeze_Layers(incep, unfreeze)

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

incep4_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep4_model_results = incep4_model.fit_generator(train_generator,
                                           class_weight = multi_weights,       
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

## **Model with Inception Pretrained Network using a smaller learning rate**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep3_model = keras.Sequential()
incep3_model.add(incep)
incep3_model.add(layers.Flatten())
incep3_model.add(layers.Dense(512, activation='relu'))
incep3_model.add(layers.BatchNormalization())
incep3_model.add(layers.Dropout(0.3))
incep3_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep3_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep3_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

small_lr_adam = keras.optimizers.Adam(learning_rate=0.0001)

incep3_model.compile(optimizer= small_lr_adam,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep3_model_results = incep3_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep3_model_results)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 94% and a testing acuracy of 76%, with a training loss of 17% and a testing loss of around 99%; the model is still overfitting. Tetsing recall is 71%. Other than the testing loss being slightly lower in this iteration than the last, the numbers between this iteration and the ast are fairly similar; it is possible this model could improve with more training epochs. It is also possible that unfreezing more layers from the pretrained base might help the network better learn and adjust to these specific images.

## **Unfreezing more layers of the Pretrained InceptionV3 Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep4_model = keras.Sequential()
incep4_model.add(incep)
incep4_model.add(layers.Flatten())
incep4_model.add(layers.Dense(512, activation='relu'))
incep4_model.add(layers.BatchNormalization())
incep4_model.add(layers.Dropout(0.3))
incep4_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep4_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep4_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93') |  (layer.name == 'conv2d_85') \
    | (layer.name == 'conv2d_92') | (layer.name == 'conv2d_91'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
incep.summary()

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

small_lr_adam = keras.optimizers.Adam(learning_rate=0.0001)

incep4_model.compile(optimizer= small_lr_adam,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep4_model_results = incep4_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss ad a training ccuracy of 95% and a testing accuracy of 75%, with a training loss of 16% and a testing loss of 108%. Testing recall is 75%. the testing loss is slightly higher than the last model iteration, but other than that the results are pretty similar. It looks like unfreezing a few more layers did not help much; it could be that unfreezing more layers wil not help, but it is also possible that since there are so many layers in this pretrained network, that a lot of layers need to be unfrozen. In the next model iteration I will try unfreezing a few more layers and changing the opimizer to SGD with momentum, since this is an optimizer well known to get good results.

## **Using SGD with momentum as an Optimizer on pretrained InceptionV3 Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep5_model = keras.Sequential()
incep5_model.add(incep)
incep5_model.add(layers.Flatten())
incep5_model.add(layers.Dense(512, activation='relu'))
incep5_model.add(layers.BatchNormalization())
incep5_model.add(layers.Dropout(0.3))
incep5_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep5_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep5_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93') |  (layer.name == 'conv2d_85') \
    | (layer.name == 'conv2d_92') | (layer.name == 'conv2d_91'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

sgd_momen = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

incep5_model.compile(optimizer= sgd_momen,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep5_model_results = incep5_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep5_model_results)

**Analysis of Model**

The epoch with the lowest loss had a training accuracy of 95% and a testing acuracy of 76%, with a training loss of 15% and a testing loss of 105%. Testing recall is 75%. The results from this model iteration are not much different from the last, meaning that using the SGD optimizer with momentum did not help in this instance. Perhaps adding a dropout layer just after the pretrained network base will improve results.

## **Adding another dropout layer to Model Using Pretrained InceptionV3 Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep6_model = keras.Sequential()
incep6_model.add(incep)
incep6_model.add(layers.Dropout(0.5))
incep6_model.add(layers.Flatten())
incep6_model.add(layers.Dense(512, activation='relu'))
incep6_model.add(layers.BatchNormalization())
incep6_model.add(layers.Dropout(0.3))
incep6_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep6_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep6_model.trainable_weights))

In [ ]:
incep.summary()

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93') |  (layer.name == 'conv2d_85') \
    | (layer.name == 'conv2d_92') | (layer.name == 'conv2d_91') | \
      (layer.name == 'conv2d_88') | (layer.name == 'conv2d_87') | \
      (layer.name == 'conv2d_90') | (layer.name == 'conv2d_86'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
# Making early stop for model
early_stop2 = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

sgd_momen = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

incep6_model.compile(optimizer= sgd_momen,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep6_model_results = incep6_model.fit(x=train_img, y=train_lab,
                                        batch_size=32,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2700//32+1,# number of samples / batch size
                                         epochs=25,
                                        callbacks= early_stop2,
                                         validation_data=(test_img, test_lab),
                                        validation_steps= 394//32+1)

**Analysis of Model**

The eposhc with the lowest testing loss had a training accuracy of 95% and a testing accuracy of 78%, with a training loss of 15% and a testing loss of 107%. Testing recall is 78%.

## **Trying out EfficientNetB4 Pretrained Network**

In [ ]:
from keras.applications.efficientnet import EfficientNetB4
eff_b4 = EfficientNetB4(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
effb4_model = keras.Sequential()
effb4_model.add(eff_b4)
effb4_model.add(layers.Flatten())
effb4_model.add(layers.Dense(500, activation='relu'))
effb4_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
Freeze_Pretrained_Base(eff_b4, effb4_model)

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

small_adam = keras.optimizers.Adam(learning_rate=0.0001)
effb4_model.compile(optimizer= small_adam,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

effb4_model_results = effb4_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=30,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model** 

The epoch with the lowest testing loss had a training accuracy of 27% and a testing accuracy of 27%, with a training loss of 203% and a testing loss of 140%. Testing recall is 0%. It is air to say that this model performed abysmally! 

In [ ]:
from keras.applications.efficientnet import EfficientNetB1
eff_b1 = EfficientNetB1(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
two_effb1 = keras.Sequential()
two_effb1.add(eff_b1)
two_effb1.add(layers.GlobalAveragePooling2D())
two_effb1.add(layers.Dropout(0.45))
two_effb1.add(layers.Dense(4, activation='softmax'))

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=2,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.00001)


In [ ]:
Freeze_Pretrained_Base(eff_b1, two_effb1)

In [ ]:
multi_weights = {0: 2,
                1: 1 ,
                2: 1,
                3: 1,}

#sgd_momen = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
two_effb1.compile(optimizer= 'adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

two_effb1_results = two_effb1.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699// 20+1,# number of samples / batch size
                                         epochs=20,
                                        callbacks= [early_stop2, reduce_lr],
                                         validation_data=test_generator,
                                           validation_steps=394//20+1)